In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score,classification_report
import sklearn
from sklearn.dummy import DummyClassifier

In [36]:
import sklearn
from sklearn.datasets import load_svmlight_file
X_test, y_test = load_svmlight_file("Test_labeledBow.feat")
X_train, y_train = load_svmlight_file("Train_labeledBow.feat")

In [37]:
import scipy.sparse._sparsetools
import scipy
X_test_df=scipy.sparse.lil_matrix(X_test, dtype='uint16')
X_train_df=scipy.sparse.lil_matrix(X_train, dtype='uint16')

In [4]:
X_test_df=X_test_df.toarray()

In [38]:
X_train_df=X_train_df.toarray()

In [40]:
X_train_df=X_train_df.sum(axis=1)

In [41]:
X_train_df

array([138, 114, 115, ..., 231, 153, 138], dtype=uint32)

In [42]:
data2=np.where(X_train_df < 17, 0, 1)

In [43]:
data2

array([1, 1, 1, ..., 1, 1, 1])

In [7]:
X_train_df=pd.DataFrame(X_train_df)

In [8]:
X_train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,89517,89518,89519,89520,89521,89522,89523,89524,89525,89526
0,9,1,4,4,6,4,2,2,4,0,...,0,0,0,0,0,0,0,0,0,0
1,7,4,2,2,0,4,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
2,4,4,4,7,2,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,10,2,2,0,3,2,4,2,0,1,...,0,0,0,0,0,0,0,0,0,0
4,13,9,6,4,2,5,10,6,0,2,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df=X_train_df[:6000]

In [21]:
df=df.sum(axis=1)

In [22]:
df.head(5)

0    138
1    114
2    115
3    104
4    232
dtype: int64

In [228]:
data = pd.DataFrame(df, columns = ["Text Length"])

In [229]:
data['Label']=data["Text Length"].apply(lambda x: '1' if x>17 else '0')

In [230]:
data=data.drop(columns=['Text Length'])
data.head(5)

,Label
0,1
1,1
2,1
3,1
4,1


In [231]:
data['Label'] = pd.to_numeric(data['Label'])

In [232]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 1 columns):
Label    6000 non-null int64
dtypes: int64(1)
memory usage: 47.0 KB


In [205]:
#print(max(df), min(df), st.mean(df))

In [45]:
y_test1= np.where(y_test <= 4, 0, 1)
y_train1=np.where(y_train <= 4, 0, 1)
label=y_train1

In [28]:
label=pd.DataFrame(label, columns = ["Target"])
label.head(5)

,Target
0,1
1,1
2,1
3,1
4,1


In [24]:
label['Target'] = pd.to_numeric(label['Target'])
label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 1 columns):
Target    6000 non-null int32
dtypes: int32(1)
memory usage: 23.5 KB


In [46]:
print("Precision: %0.2f" %precision_score(label, data2, average="macro"))
print("Recall:  %0.2f" %recall_score(label, data2, average="macro"))
print("F1-score:  %0.2f" %f1_score(label, data2 , average="macro"))

Precision: 0.68
Recall:  0.50
F1-score:  0.33


In [47]:
get_precision(label, data2, debug = False)


0.99992

In [185]:
get_recall(data, label)

0

In [15]:
#get_fscore(data, label)

In [14]:
def get_tp_tn_fn_fp(y_test1, y_pred):

    tp = sum((y_test1 == 1) & (y_pred == 1))
    tn = sum((y_test1 == 0) & (y_pred == 0))
    fn = sum((y_test1 == 1) & (y_pred == 0))
    fp = sum((y_test1 == 0) & (y_pred == 1))
    return tp, tn, fp, fn

In [13]:
## Calculates the precision of the predicted labels
def get_precision(y_pred, y_true, debug = False):
    # deal with npdarray
    y_pred = list(y_pred)
    y_true = list(y_true)
    #---------
    n = len(y_pred);
    y_pred = [0 if v is None else v for v in y_pred]# deal with None type
    y_true = [0 if v is None else v for v in y_true]# deal with None type
    true_positive = sum(y_pred[i]* y_true[i] for i in range(n))
    if (0 == sum(y_pred)): return 0
    return true_positive*1.0/sum(y_pred)
    
## Calculates the recall of the predicted labels
def get_recall(y_pred, y_true):
    # deal with npdarray
    y_pred = list(y_pred)
    y_true = list(y_true)
    #---------
    n = len(y_pred);
    y_pred = list(map(int,[1 == l for l in y_pred]))# deal with None type
    y_true = list(map(int,[1 == l for l in y_true]))# deal with None type
    true_positive = sum(y_pred[i]*y_true[i] for i in range(n))
    if 0 == sum(y_true): return 0
    return true_positive*1.0/sum(y_true)
    

## Calculates the f-score of the predicted labels
def get_fscore(y_pred, y_true):
    precision = get_precision(y_pred, y_true);
    if (0 == precision): return 0
    recall= get_recall(y_pred, y_true);
    if (0 == recall): return 0
    beta = 1.0;
    # print("get_fscore:",(beta**2*precision+recall))
    fscore = (beta**2+1)*precision*recall/(beta**2*precision+recall);
    return fscore

In [12]:
def word_length_threshold(df, label, plot_flag = True):
    ## YOUR CODE HERE
    # print("load training file")
    #words,labels = load_file(training_file, True)
    Dict_len = defaultdict(set)
    for w in df: Dict_len[w].add(w)
    # Evaluation depending on fscore
    Min = min(Dict_len)
    Max = max(Dict_len)
    #for Thres in range(Min,Max): 
            #print(list(map(int,[w < Thres for w in df])))
            #print(get_precision(list(map(int,[w < Thres for w in df])), label ) ) 
    #----------
    #yezheng: binary search is possible to improve the serach
    #plotting precision vs. recall 
    ThreRange=  range(Min,Max+1)
    FscoreL = dict([(f1_score(list(map(int,[w > Thres for w in df])), label, average="macro"),Thres) for Thres in ThreRange])
    Thres_opt = FscoreL[max(FscoreL)]
    if plot_flag:
         print("Range of thresholds:",Min,"to",Max, " with optimal threshold:", Thres_opt+1)
    #precisionL = [precision_score(list(map(int,[w > Thres for w in df])), label,average="macro")for Thres in ThreRange]
    #recallL = [recall_score(list(map(int,[w > Thres for w in df])), label, average="macro") for Thres in ThreRange]   
    #Pred_opt = list(map(int,[w> Thres_opt for w in df]))
    training_performance = [precision_score(Pred_opt, label, average="macro"), recall_score(Pred_opt, label, average="macro" ), f1_score(Pred_opt, label, average="macro" )]
    del df
    del label
    return ThreRange, Thres_opt, training_performance

In [12]:
word_length_threshold(df, label)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Range of thresholds: 12 to 2476  with optimal threshold: 17


NameError: name 'Pred_opt' is not defined

In [ ]:
### 2.2: Word length thresholding

## Finds the best length threshold by f-score, and uses this threshold to
## classify the training and development set
def word_length_threshold(training_file, development_file, plot_flag = False):
    ## YOUR CODE HERE
    # print("load training file")
    words,labels = load_file(training_file, True)
    Dict_len = defaultdict(set)
    for w in words: Dict_len[len(w)].add(w)
    # Evaluation depending on fscore
    Min = min(Dict_len)
    Max = max(Dict_len)
    # for Thres in range(Min,Max): 
    #     # print(list(map(int,[len(w) < Thres for w in words])))
    #     print(get_precision(list(map(int,[len(w) < Thres for w in words])), labels ) ) 
    #----------
    #yezheng: binary search is possible to improve the serach
    #plotting precision vs. recall 
    ThreRange =  range(Min,Max+1)
    FscoreL = dict([(get_fscore(list(map(int,[len(w) > Thres for w in words])), labels),Thres) for Thres in ThreRange])
    Thres_opt = FscoreL[max(FscoreL)]
    if plot_flag:
        print("Range of thresholds:",Min,"to",Max, " with optimal threshold:", Thres_opt+1)
        precisionL = [get_precision(list(map(int,[len(w) > Thres for w in words])), labels)for Thres in ThreRange]
        recallL = [get_recall(list(map(int,[len(w) > Thres for w in words])), labels) for Thres in ThreRange]
        Ret_plot = [precisionL,recallL]
        plt.plot(recallL, precisionL,'^r', label = 'Train')
        plt.title('word_length_threshold: precision-recall curve')
        plt.xlabel('Recall')
        plt.ylabel('Precision')

    Pred_opt = list(map(int,[len(w)> Thres_opt for w in words]))
    training_performance = [get_precision(Pred_opt, labels), get_recall(Pred_opt, labels), get_fscore(Pred_opt, labels)]
    del words
    del labels
    # print("load development file")
    words,labels = load_file(development_file)
    Pred_opt = list(map(int,[len(w)> Thres_opt for w in words]))
    if plot_flag: 
        precisionL = [get_precision(list(map(int,[len(w) > Thres for w in words])), labels)for Thres in ThreRange]
        recallL = [get_recall(list(map(int,[len(w) > Thres for w in words])), labels) for Thres in ThreRange]
        plt.plot(recallL, precisionL,'^g', label = 'Dev')
        plt.legend(loc='upper right')
        plt.show()
        Ret_plot += [precisionL,recallL]
    development_performance = [get_precision(Pred_opt, labels), get_recall(Pred_opt, labels), get_fscore(Pred_opt, labels)]
    if plot_flag: return training_performance, development_performance,Ret_plot
    return training_performance, development_performance